In [18]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [76]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [62]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [21]:
#import random
#random.shuffle(spDaily)

In [22]:
#trainLen = int(len(spDaily)*0.75)

In [23]:
#trainData=spDaily[:trainLen]
#testData=spDaily[trainLen:]

In [65]:
tempData=spDaily[:1]

# Creating the Algorithim

In [66]:
algoTools = AlgoTools(tickData)

In [67]:
def Algo(data,r, a_L, a_S, E_e, stopProf, stopLoss):
    #data = tempData
   # r, a_L, a_S, E_e, stopProf, stopLoss = params
    trader = Trader()
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2330<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            elif (algoTools.getAccel(t) >= a_L ) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:],"A_L reached")
            
            #same as previous statment but for shorting
            #elif  ( algoTools.getAccel(t) <= a_S) and trader.getLastTradePos() == 0:
            #    trader.goShort(data[d].iloc[index,:],"A_S reached")
                
            
            
            
            
           
            
            #-----Closing Conditions------
            
            
            #as per calclus reasoing if the first deravitive is = 0 it has found a local extrema. therfore close out 
            #E_e is error of extrema
          
            elif algoTools.getRate(t) <= E_e and trader.getLastTradePos() != 0:
                trader.closeOut(data[d].iloc[index,:],"E_e reached")
                
#             elif trader.getLastTradePos() == 1 and algoTools.getAccel(t) <a_S:
#                 trader.closeOut(data[d].iloc[index,:],"Acel Switched")
            
#             elif trader.getLastTradePos() == -1 and algoTools.getAccel(t) > a_L:
#                 trader.closeOut(data[d].iloc[index,:],"Acel Switched")
               
            
            #stop profit (in points)
            elif (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:],"Stop Prof reached")
                
            
            #stop loss (in points)
            elif trader.getLastTradePos() == 1 and (trader.getLastTradePrice()-data[d].loc[t,"price"] ) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached L")
                
                
            elif trader.getLastTradePos() == -1 and (data[d].loc[t,"price"]-trader.getLastTradePrice()) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached S")
                


            
                
            
            
                
            
    return trader

In [68]:
trader = Algo(tempData,1,0.4,-0.4,0.005,0.5,0.5)

In [69]:
trader.getTradeInfo()

{'Total Return': 29.999999999972715,
 'Max Return': 29.999999999995453,
 'Min Return': -25.0}

In [70]:
trader.getLR()

0    0.534091
dtype: float64

In [71]:
tradeData = trader.getTradeData()
tradeData

,Position,TradePrice,TradeTime,Method
0,1.0,1495.5,2000-01-03 08:31:25,A_L reached
1,0.0,1495.0,2000-01-03 08:31:26,E_e reached
2,1.0,1492.5,2000-01-03 08:33:56,A_L reached
3,0.0,1492.0,2000-01-03 08:34:00,E_e reached
4,1.0,1491.5,2000-01-03 08:34:23,A_L reached
...,...,...,...,...
181,0.0,1469.0,2000-01-03 14:59:10,Stop Prof reached
182,1.0,1468.5,2000-01-03 14:59:30,A_L reached
183,0.0,1469.0,2000-01-03 14:59:35,Stop Prof reached
184,1.0,1468.0,2000-01-03 15:07:21,A_L reached


In [72]:
def multiLoss(trader):
    lossRatio = trader.getLR()
    profit = trader.getTradeInfo()["Total Return"] 
    avg_profit = profit/(len(trader.getTradeData())/2) # profit divided by number of hlaf the number of trades( half of the trades are just closing out)
    
    print(lossRatio, profit,avg_profit)
    

In [73]:
multiLoss(trader)

0    0.534091
dtype: float64 29.999999999972715 0.32258064516099694


In [74]:
fig = px.line(tempData[0], x='Time2', y='price', title='Price Over Time', labels={'Time': 'Time', 'price': 'Price'})
scatter = go.Scatter(
    x=tradeData['TradeTime'], 
    y=tradeData['TradePrice'], 
    mode='markers', 
    marker=dict(color='red'), 
    name='Trade Points',
    text=tradeData['Method']  # Add labels from the 'Method' column
    #textposition='top center'  # Position of the text labels
)
# Add scatter plot to the line plot
fig.add_trace(scatter)

fig.show()

In [85]:
dailyConCat= pd.DataFrame()
for x in spDaily:
    dailyConCat=  pd.concat([dailyConCat,x])
    

In [86]:
dailyConCat

,date,time,price,volume,Time2
0,01/03/2000,08:30:34.000,1496.4,0,2000-01-03 08:30:34.000
1,01/03/2000,08:30:36.000,1496.0,0,2000-01-03 08:30:36.000
2,01/03/2000,08:30:37.000,1495.5,0,2000-01-03 08:30:37.000
3,01/03/2000,08:30:46.000,1495.0,0,2000-01-03 08:30:46.000
4,01/03/2000,08:30:53.000,1495.5,0,2000-01-03 08:30:53.000
...,...,...,...,...,...
13157507,06/24/2013,23:54:14.423,1563.0,1,2013-06-24 23:54:14.423
13157508,06/24/2013,23:54:14.423,1563.0,1,2013-06-24 23:54:14.423
13157509,06/24/2013,23:54:14.423,1563.0,3,2013-06-24 23:54:14.423
13157510,06/24/2013,23:54:14.423,1563.0,2,2013-06-24 23:54:14.423


In [77]:
spDaily

[            date          time   price  volume               Time2
 0     01/03/2000  08:30:34.000  1496.4       0 2000-01-03 08:30:34
 1     01/03/2000  08:30:36.000  1496.0       0 2000-01-03 08:30:36
 2     01/03/2000  08:30:37.000  1495.5       0 2000-01-03 08:30:37
 3     01/03/2000  08:30:46.000  1495.0       0 2000-01-03 08:30:46
 4     01/03/2000  08:30:53.000  1495.5       0 2000-01-03 08:30:53
 ...          ...           ...     ...     ...                 ...
 4329  01/03/2000  15:14:20.000  1467.6       0 2000-01-03 15:14:20
 4330  01/03/2000  15:14:23.000  1467.7       0 2000-01-03 15:14:23
 4331  01/03/2000  15:14:23.000  1467.8       0 2000-01-03 15:14:23
 4332  01/03/2000  15:14:25.000  1467.5       0 2000-01-03 15:14:25
 4333  01/03/2000  15:14:30.000  1467.0       0 2000-01-03 15:14:30
 
 [4334 rows x 5 columns],
             date          time   price  volume               Time2
 4334  01/04/2000  08:30:08.000  1451.0       0 2000-01-04 08:30:08
 4335  01/04/2000  0

In [82]:
spDaily[1]

date               01/04/2000
time             08:30:08.000
price                  1451.0
volume                      0
Time2     2000-01-04 08:30:08
Name: 4334, dtype: object